In [74]:
import numpy as np
import h5py
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import SimpleRNN,LSTM, Dense, Activation
from keras.utils import to_categorical
#from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

In [81]:
A01T = h5py.File('datasets/A01T_slice.mat','r')
data = np.copy(A01T['image'])
data = np.transpose(data,(0,2,1))
data = data[:,:,:22]
labels = np.copy(A01T['type'])
labels = labels[0,0:data.shape[0]:1]
labels = np.asarray(labels, dtype=np.int32)

a = data[:56]
b = data[57:]
data = np.vstack((a,b))
a = labels[:56]
b = labels[57:]
labels = np.hstack((a,b))
#enc = OneHotEncoder()
#enc_labels = enc.fit_transform(labels.reshape(-1,1)).toarray()
enc_labels = to_categorical(labels-769, num_classes=4)
print(enc_labels)

#scaler = StandardScaler()
#data = scaler.fit_transform(data,enc_labels)

[[ 0.  0.  0.  1.]
 [ 0.  0.  1.  0.]
 [ 0.  1.  0.  0.]
 ..., 
 [ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 1.  0.  0.  0.]]


In [53]:
model = Sequential([
    LSTM(100, input_shape=(1000, 22)),
    Dense(32),
    Activation('relu'),
    Dense(64),
    Activation('relu'),
    Dense(32),
    Activation('relu'),
    Dense(4),
    Activation('softmax'),
])

bs, t, f = data.shape

In [54]:
model.layers

In [55]:
shuffle = np.random.choice(bs,bs,replace=False)

train_samples = 200
train_data = data[:train_samples,:,:]
train_labels = enc_labels[:train_samples]
test_data = data[train_samples:,:,:]
test_labels =enc_labels[train_samples:]

model.compile(optimizer = 'adam',
             loss = 'categorical_crossentropy',
             metrics=['accuracy'])

hist = model.fit(train_data,train_labels,epochs=5,validation_split=0.25,batch_size=32)
test_score = model.evaluate(test_data, test_labels, batch_size=32)


Train on 150 samples, validate on 50 samples
Epoch 1/5
150/150 [==============================] - 6s 37ms/step - loss: 1.4017 - acc: 0.2400 - val_loss: 1.4041 - val_acc: 0.2000
Epoch 2/5
150/150 [==============================] - 7s 45ms/step - loss: 1.3212 - acc: 0.4267 - val_loss: 1.4078 - val_acc: 0.2400
Epoch 3/5
150/150 [==============================] - 8s 54ms/step - loss: 1.2706 - acc: 0.5400 - val_loss: 1.4198 - val_acc: 0.2200
Epoch 4/5
150/150 [==============================] - 9s 59ms/step - loss: 1.2177 - acc: 0.5867 - val_loss: 1.4118 - val_acc: 0.2600
Epoch 5/5
87/87 [==============================] - 1s 9ms/step


In [56]:
test_score

[1.4515078245908364, 0.22988505644359808]

Modified VGG net to handle our input (need to check dimensions and might need to transpose input to original shape)

In [72]:
# VGGnet
# https://keras.io/getting-started/sequential-model-guide/#examples

import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv1D, MaxPooling1D, BatchNormalization
from keras.optimizers import SGD

del model
model = Sequential()
#model.add(LSTM(100, input_shape=(1000, 22)))
model.add(Conv1D(512, 3, activation='relu',input_shape=(1000, 22)))             #Originally 32 each
model.add(BatchNormalization())
model.add(Conv1D(512, 3, activation='relu'))
model.add(MaxPooling1D())
model.add(Dropout(0.25))

model.add(Conv1D(64, 3, activation='relu'))                                    #Originally 64 each
model.add(BatchNormalization())
model.add(Conv1D(64, 3, activation='relu'))
model.add(MaxPooling1D())
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))

#sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
#model.compile(loss='categorical_crossentropy', optimizer=sgd)

model.compile(optimizer = 'adam',
             loss = 'categorical_crossentropy',
             metrics=['accuracy'])

hist = model.fit(train_data,train_labels,epochs=5,validation_split=0.25,batch_size=32)
test_score = model.evaluate(test_data, test_labels, batch_size=32)

Train on 150 samples, validate on 50 samples
Epoch 1/5
150/150 [==============================] - 30s 198ms/step - loss: 8.5472 - acc: 0.2467 - val_loss: 10.9118 - val_acc: 0.2800
Epoch 2/5
150/150 [==============================] - 25s 167ms/step - loss: 9.3925 - acc: 0.3467 - val_loss: 11.0357 - val_acc: 0.2800
Epoch 3/5
150/150 [==============================] - 24s 161ms/step - loss: 9.6391 - acc: 0.3467 - val_loss: 12.2498 - val_acc: 0.2400
Epoch 4/5
150/150 [==============================] - 25s 164ms/step - loss: 10.3226 - acc: 0.3200 - val_loss: 10.6407 - val_acc: 0.3200
Epoch 5/5
87/87 [==============================] - 5s 55ms/step


In [68]:
test_score

[12.227520537102359, 0.24137930931716128]

Simple RNN model

In [71]:
model = Sequential([
    SimpleRNN(64, input_shape=(t,f)),
    Dense(32),
    BatchNormalization(),
    Activation('relu'),
    Dense(4),
    Activation('softmax'),
])

model.compile(optimizer = 'adam',
             loss = 'categorical_crossentropy',
             metrics=['accuracy'])

hist = model.fit(train_data,train_labels,epochs=10,validation_split=0.25,batch_size=32)
test_score = model.evaluate(test_data, test_labels, batch_size=32)

Train on 150 samples, validate on 50 samples
Epoch 1/10
150/150 [==============================] - 2s 15ms/step - loss: 1.6141 - acc: 0.2933 - val_loss: 1.6435 - val_acc: 0.2800
Epoch 2/10
150/150 [==============================] - 1s 9ms/step - loss: 1.4375 - acc: 0.3467 - val_loss: 1.6217 - val_acc: 0.2600
Epoch 3/10
150/150 [==============================] - 1s 9ms/step - loss: 1.3728 - acc: 0.3467 - val_loss: 1.6016 - val_acc: 0.2600
Epoch 4/10
150/150 [==============================] - 1s 8ms/step - loss: 1.3013 - acc: 0.3667 - val_loss: 1.5964 - val_acc: 0.2600
Epoch 5/10
150/150 [==============================] - 1s 9ms/step - loss: 1.2397 - acc: 0.4467 - val_loss: 1.5811 - val_acc: 0.2600
Epoch 6/10
150/150 [==============================] - 1s 10ms/step - loss: 1.1887 - acc: 0.4867 - val_loss: 1.5686 - val_acc: 0.2200
Epoch 7/10
150/150 [==============================] - 1s 9ms/step - loss: 1.1641 - acc: 0.5000 - val_loss: 1.5548 - val_acc: 0.2200
Epoch 8/10
150/150 [=========

In [15]:
test_score

[1.3988697637211194, 0.23863636363636365]